In [1]:
#include <chrono>
#include <ctime>
#include <cmath>
#include <algoviz/SVG.hpp>
#include "Object.hpp"
#include "ScrollingImage.hpp"
#include "Player.hpp"
#include "ObstacleManager.hpp"
#include "Bread.hpp"

using namespace std;

AlgoViz::clear();

/******************
 * Anzeigeobjekte *
 ******************/
SVG canvas = SVG(400, 400, 20, 20, "Projekt");
ScrollingImage backgroundWater = ScrollingImage("sprites/background.png", 20, canvas.getHeight() * 0.9, &canvas);

Player player = Player(200, 200, &canvas);

Storm leftBorder = Storm(&canvas, nullptr);
ObstacleManager obstacleManager = ObstacleManager(&canvas, &leftBorder);

Bread bread = Bread(&leftBorder, &player, &canvas);

ScrollingImage upperShore = ScrollingImage("sprites/shore.png", 0, canvas.getHeight() * 0.05, &canvas);
ScrollingImage lowerShore = ScrollingImage("sprites/shore.png", 380, canvas.getHeight() * 0.05, &canvas);

Text fps = Text("", 2, 20, &canvas);

//Spielverwaltung
bool continueGame = true;
string button = "";
double deltaTime = 0.05;
double totalGameTime = 0;
double noObstacleTime = 0;

In [2]:
void updateGame(string button, double deltaTime) {
    //Objektupdates
    player.update(button, deltaTime);
    obstacleManager.updateAll(deltaTime);
    bread.update(deltaTime);
    backgroundWater.move(-90 * deltaTime);
    upperShore.move(-10 * deltaTime);
    lowerShore.move(-10 * deltaTime);
    
    //Zeitupdates
    noObstacleTime -= deltaTime;
    totalGameTime += deltaTime;
    
    //Hindernisse erscheinen lassen
    if(noObstacleTime <= 0 && rand() % 100 < (30.0 + 0.3 * totalGameTime) * deltaTime) {
        obstacleManager.addTrunk();
        noObstacleTime = 0.4;
    }
    
    //Spiel beenden
    if(obstacleManager.hasCollidingObject(&player) || button == "q") {
        continueGame = false;
    }
}

In [4]:
?Circle

In [3]:
AlgoViz::sleep(500);
while (continueGame) {
    /*
     * Die Dauer des vorherigen Schleifendurchlaufs
     * wird als Referenz dafür verwendet, wieviel
     * sich Objekte bewegen sollen.
     * Dadurch bewegen sie sich pro Sekunde immer
     * ungefähr gleich schnell, unabhängig von der
     * Dauer der Schleifendurchläufe.
     */
    auto start = chrono::system_clock::now();
    button = canvas.lastKey();
    
    fps.setText(to_string((int)(1 / deltaTime)) + " FPS");
    
    updateGame(button, deltaTime);
    
    auto end = std::chrono::system_clock::now();
    //Dauer des Schleifendurchlaufs ausrechnen und speichern
    chrono::duration<double> elapsed_seconds = end-start;
    deltaTime = elapsed_seconds.count();
}
fps.setText("Total Time: " + to_string((int)(totalGameTime)) + " Seconds");